In [ ]:
from os import getenv
from pathlib import Path
output_folder = Path(getenv(
    'CROSSCOMPUTE_OUTPUT_FOLDER', 'batches/standard/output'))
output_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
datasets_folder = Path('datasets')
runs_folder = datasets_folder / 'runs'

In [ ]:
import pandas as pd
import sys
cell_towers_path = datasets_folder / 'cell-towers.csv'
try:
    cell_towers_table = pd.read_csv(cell_towers_path)
except OSError:
    sys.stderr.write(f'could not open {cell_towers_path}')

In [ ]:
import json
entries = []
for run_folder in runs_folder.glob('*'):
    path = run_folder / 'input' / 'variables.dictionary'
    with path.open('rt') as f:
        entries.append(json.load(f))
len(entries)

In [ ]:
features = []
features.extend({
    'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': _},
    'properties': {},
} for _ in cell_towers_table[['X', 'Y']].values.tolist())
features.extend({
    'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': _['location']['center']},
    'properties': {'is_contributed': 1},
} for _ in entries)

In [ ]:
with (output_folder / 'locations.geojson').open('wt') as f:
    json.dump({'type': 'FeatureCollection', 'features': features}, f)